# sscan as Bluesky plan

## 1D step scans using sscan record

Support the [sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html) with a  [bluesky](http://nsls-ii.github.io/bluesky) plan for data acquisition.  Consider the case of [1D step scans using sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html#HEADING_1-1).

In [1]:
# Import matplotlib and put it in interactive mode.
%matplotlib notebook
import matplotlib.pyplot as plt
plt.ion()

from collections import OrderedDict
import time

# common IOC prefix to be used
P = "gp:"

In [2]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH(f"{P}scaler1", name="scaler")
scaler.select_channels(None)

In [3]:
from ophyd import EpicsMotor
m1 = EpicsMotor(f"{P}m1", name="m1")

In [4]:
from apstools.synApps import UserCalcsDevice
calcs = UserCalcsDevice(P, name="calcs")

In [5]:
from apstools.synApps import SscanDevice
scans = SscanDevice(P, name="scans")
scans.select_channels()

In [6]:
from apstools.synApps import SaveData
save_data = SaveData(f"{P}saveData_", name="save_data")

In [7]:
# configure saveData for data collection into MDA files:
        
save_data.file_system.put("/tmp")
save_data.subdirectory.put("saveData")
save_data.base_name.put("sscan1_")
save_data.next_scan_number.put(1)
save_data.comment1.put("testing")
save_data.comment2.put("configured and run from ophyd")

In [8]:
# configure the sscan record for data collection:

# clear out the weeds
scans.reset()

scan = scans.scan1
scan.number_points.put(6)
scan.positioners.p1.setpoint_pv.put(m1.user_setpoint.pvname)
scan.positioners.p1.readback_pv.put(m1.user_readback.pvname)
scan.positioners.p1.start.put(-1)
scan.positioners.p1.end.put(0)
scan.positioner_delay.put(0.0)
scan.detector_delay.put(0.1)
scan.detectors.d01.input_pv.put(scaler.channels.chan03.s.pvname)
scan.detectors.d02.input_pv.put(scaler.channels.chan02.s.pvname)
scan.triggers.t1.trigger_pv.put(scaler.count.pvname)

# finally, reconfigure
scans.select_channels()

In [9]:
# make a noisy detector in an EPICS swait record, peak ceneter at 2
from apstools.synApps import setup_lorentzian_swait
setup_lorentzian_swait(calcs.calc2, m1.user_readback, 2)
noisy_det = calcs.calc2.calculated_value
noisy_det.kind = "hinted"

In [10]:
def ophyd_step_scan(motor):
    """step-scan the motor and read the noisy detector"""
    t0 = time.time()
    for p in range(10):
        motor.move(p-3)
        print(
            "%8.3f" % (time.time()-t0), 
            "%8.2f" % motor.position, 
            "%8.4f" % noisy_det.get()
             )
    motor.move(0)
    print("Complete in %.3f seconds" % (time.time()-t0))

# ophyd_step_scan(m1)

--------
## setup Bluesky, databroker, and the RunEngine

In [11]:
import databroker
cat = databroker.temp()

In [12]:
from bluesky import RunEngine
import bluesky.plan_stubs as bps
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky import SupplementalData

RE = RunEngine({})
RE.subscribe(cat.v1.insert)
RE.subscribe(BestEffortCallback())
RE.preprocessors.append(SupplementalData())

------
## Develop the BS plan

In [13]:
import ophyd

DIAGNOSTIC_PRINTING = False


def get_sscan_data_objects(sscan):
    """
    prepare a dictionary of the "interesting" ophyd data objects for this scan
    """
    scan_data_objects = OrderedDict()
    for part in (sscan.positioners, sscan.detectors):
        for chname in part.read_attrs:
            if not chname.endswith("_value"):
                continue
            obj = getattr(part, chname)
            key = obj.name.lstrip(sscan.name + "_")
            scan_data_objects[key] = obj
    return scan_data_objects

    
def sscan_step_1D(sscan, _md={}):
    """
    simple 1-D step scan using EPICS synApps sscan record
    
    assumes the sscan record has already been setup properly for a scan
    """
    global new_data
    
    t0 = time.time()
    sscan_status = ophyd.DeviceStatus(sscan.execute_scan)
    started = False
    new_data = False
    
    def execute_cb(value, timestamp, **kwargs):
        """watch for sscan to complete"""
        if DIAGNOSTIC_PRINTING:
            elapsed = "%.3f" % (time.time() - t0)
            phase = sscan.scan_phase.get(as_string=True)
            print(
                f"{elapsed} execute_cb()"
                f" {timestamp}:"
                f" phase={phase}"
                f" executing={value}"
            )
        if started and value in (0, "IDLE"):
            sscan_status._finished()
            sscan.execute_scan.unsubscribe_all()
            sscan.scan_phase.unsubscribe_all()
    
    def phase_cb(value, timestamp, **kwargs):
        """watch for new data"""
        global new_data
        if DIAGNOSTIC_PRINTING:
            elapsed = "%.3f" % (time.time() - t0)
            phase = sscan.scan_phase.enum_strs[value]
            print(
                f"{elapsed} phase_cb()"
                f" {timestamp}:"
                f" phase={phase}"
                f" value={value}"
            )
        if value in (15, "RECORD SCALAR DATA"):
            new_data = True            # set flag for main plan
    
    sscan.select_channels()
    sscan_data_objects = get_sscan_data_objects(sscan)
    
    sscan.execute_scan.subscribe(execute_cb)
    sscan.scan_phase.subscribe(phase_cb)

    yield from bps.open_run(_md)               # start data collection
    yield from bps.mv(sscan.execute_scan, 1)   # start sscan
    started = True

    # collect and emit data, wait for sscan to end
    while not sscan_status.done or new_data:
        if DIAGNOSTIC_PRINTING:
            elapsed = "%.3f" % (time.time() - t0)
            print(
                f"{elapsed} plan()"
                f" new data={new_data}"
                f" status={sscan_status}"
            )
        if new_data:
            new_data = False
            yield from bps.create("primary")
            for k, obj in sscan_data_objects.items():
                if DIAGNOSTIC_PRINTING:
                    print(f"obj[{k}] = {obj}")
                yield from bps.read(obj)
            yield from bps.save()
        yield from bps.sleep(0.001)

    # dump the entire sscan record into another stream
    yield from bps.create("sscan")
    yield from bps.read(sscan)
    yield from bps.save()

    yield from bps.close_run()

    return sscan_status

In [14]:
scans = SscanDevice(P, name="scans")
scans.select_channels()

RE(sscan_step_1D(scans.scan1), md=dict(purpose="development", issue="#91"))



Transient Scan ID: 1     Time: 2021-11-30 17:14:44
Persistent Unique Scan ID: 'b5e733ed-58d5-43aa-b998-a774388d4817'
New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+-------------------------------------------+-----------------------------------------+-----------------------------------------+
|   seq_num |       time | scans_scan1_positioners_p1_readback_value | scans_scan1_detectors_d02_current_value | scans_scan1_detectors_d01_current_value |
+-----------+------------+-------------------------------------------+-----------------------------------------+-----------------------------------------+
|         1 | 17:14:46.9 |                                  -1.00000 |                                   4.000 |                                   4.000 |
|         2 | 17:14:48.4 |                                  -0.80000 |                                   6.000 |                                   5.000 |
|         3 | 17:14:50.1 |                                  -0.60000 |                                   7.000 |                                   8.000 |
|         4 | 17:14:51.8 |                                  -0.40000 |

('b5e733ed-58d5-43aa-b998-a774388d4817',)

In [15]:
run = cat.v2[-1]

In [16]:
run.primary.read()

<xarray.Dataset>
Dimensions:                                    (time: 6)
Coordinates:
  * time                                       (time) float64 1.638e+09 ... 1...
Data variables:
    scans_scan1_positioners_p1_readback_value  (time) float64 -1.0 -0.8 ... 0.0
    scans_scan1_detectors_d02_current_value    (time) float64 4.0 6.0 ... 6.0
    scans_scan1_detectors_d01_current_value    (time) float64 4.0 5.0 ... 5.0
    scans_scan1_positioners_p1_setpoint_value  (time) float64 -0.8 ... -5.551...

In [17]:
run.sscan.read()

<xarray.Dataset>
Dimensions:                                    (time: 1, dim_0: 1000)
Coordinates:
  * time                                       (time) float64 1.638e+09
Dimensions without coordinates: dim_0
Data variables:
    scans_scan1_current_point                  (time) int64 6
    scans_scan1_positioners_p1_readback_value  (time) float64 0.0
    scans_scan1_positioners_p1_array           (time, dim_0) float64 -1.0 ......
    scans_scan1_positioners_p1_setpoint_value  (time) float64 -5.551e-17
    scans_scan1_detectors_d01_current_value    (time) float64 5.0
    scans_scan1_detectors_d02_current_value    (time) float64 6.0

-----------------------

## test the code added to *apstools*

In [18]:
from apstools.synApps import SscanDevice

scans = SscanDevice(P, name="scans")
# assume ready setup for this 1-D scan
RE(sscan_step_1D(scans.scan1), md=dict(purpose="demo"))



Transient Scan ID: 2     Time: 2021-11-30 17:14:56
Persistent Unique Scan ID: '5c75bd48-4810-41f5-9fd2-fec5b617b799'
New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+-----------------------------------------+-------------------------------------------+-----------------------------------------+
|   seq_num |       time | scans_scan1_detectors_d02_current_value | scans_scan1_positioners_p1_readback_value | scans_scan1_detectors_d01_current_value |
+-----------+------------+-----------------------------------------+-------------------------------------------+-----------------------------------------+
|         1 | 17:14:59.0 |                                   5.000 |                                  -1.00000 |                                   7.000 |
|         2 | 17:15:00.7 |                                   4.000 |                                  -0.80000 |                                   4.000 |
|         3 | 17:15:02.5 |                                   4.000 |                                  -0.60000 |                                   6.000 |
|         4 | 17:15:04.3 |                                  17.000 |  

('5c75bd48-4810-41f5-9fd2-fec5b617b799',)

In [19]:
run = cat.v2[-1]
run.primary.read()

<xarray.Dataset>
Dimensions:                                    (time: 6)
Coordinates:
  * time                                       (time) float64 1.638e+09 ... 1...
Data variables:
    scans_scan1_detectors_d02_current_value    (time) float64 5.0 4.0 ... 4.0
    scans_scan1_positioners_p1_readback_value  (time) float64 -1.0 -0.8 ... 0.0
    scans_scan1_positioners_p1_setpoint_value  (time) float64 -0.8 ... -5.551...
    scans_scan1_detectors_d01_current_value    (time) float64 7.0 4.0 ... 5.0

In [20]:
run.sscan.read()

<xarray.Dataset>
Dimensions:                                    (time: 1, dim_0: 1000)
Coordinates:
  * time                                       (time) float64 1.638e+09
Dimensions without coordinates: dim_0
Data variables:
    scans_scan1_current_point                  (time) int64 6
    scans_scan1_positioners_p1_readback_value  (time) float64 0.0
    scans_scan1_positioners_p1_array           (time, dim_0) float64 -1.0 ......
    scans_scan1_positioners_p1_setpoint_value  (time) float64 -5.551e-17
    scans_scan1_detectors_d01_current_value    (time) float64 5.0
    scans_scan1_detectors_d02_current_value    (time) float64 4.0